<a href="https://colab.research.google.com/github/Alirs2002/skimLit-200k-RTC-PubMed/blob/main/skimlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this project,I am going to classify pubMed 200k RCT dataset's sentences,into 5 groups

1-objective

2-methods

3-results

4-conclusions

5-background

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

now,lets get the data

In [30]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git


fatal: destination path 'pubmed-rct' already exists and is not an empty directory.


In [31]:
!ls

pubmed-rct  sample_data  train.txt


In [32]:
!cd pubmed-rct/

In [33]:
!ls pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.zip


In [34]:
data_dir = "pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign"

In [35]:
import os

file_names = [data_dir + file_name for file_name in os.listdir(data_dir)]
print(file_names)

['pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signdev.txt', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtrain.zip', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtest.txt']


In [36]:
!pwd

/content


In [37]:
import zipfile

zipfile = zipfile.ZipFile("./pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip")
zipfile.extractall()

In [38]:
!ls

pubmed-rct  sample_data  train.txt


In [44]:
def read_lines(filename):
  listt = []
  with open(filename,"r") as f:
    for line in f:
      listt.append(line)

  return listt


In [46]:
lines = read_lines("train.txt")
print(len(lines))

2593169


now,it is time to create a preprocessing function

In [47]:
def preprocess(filename):

  input_lines = read_lines(filename)

  abstract_lines = ""
  abstract_samples = []


  for line in input_lines:
    if line.startswith("###"):
      abstract_id = line
      abstract_lines = ""
    elif line.isspace():
      abstract_line_split = abstract_lines.splitlines()

      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {}
        target_text_split = abstract_line.split("\t")
        line_data["target"] = target_text_split[0]
        line_data["text"] = target_text_split[1].lower()
        line_data["line_number"] = abstract_line_number
        line_data["total_lines"] = len(abstract_line_split) - 1
        abstract_samples.append(line_data)

    else:
      abstract_lines += line
  return abstract_samples

In [48]:
test_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.txt")
train_sample_data = preprocess("/content/train.txt")
validation_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.txt")


In [50]:
train_sample_data[10:15]

[{'target': 'BACKGROUND',
  'text': 'actrn@ .',
  'line_number': 10,
  'total_lines': 10},
 {'target': 'BACKGROUND',
  'text': 'the aim of this study was to evaluate the efficacy , safety and complications of orbital steroid injection versus oral steroid therapy in the management of thyroid-related ophthalmopathy .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients suffering from thyroid ophthalmopathy were included in this study .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'patients were randomized into two groups : group i included @ patients treated with oral prednisolone and group ii included @ patients treated with peribulbar triamcinolone orbital injection .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'only @ patients in both groups ( @ female and @ male ) completed the study .',
  'line_number': 3,
  'total_lines': 11}]

as you can see,our preprocessing function is working great

In [54]:
# now,lets build a pandas dataframe,for better preformance

In [56]:
len(train_sample_data),len(test_sample_data),len(validation_sample_data)

(2211861, 29493, 28932)

In [58]:
train_data = pd.DataFrame(train_sample_data)
test_data = pd.DataFrame(test_sample_data)
val_data = pd.DataFrame(validation_sample_data)

In [59]:
train_data

,target,text,line_number,total_lines
0,BACKGROUND,the emergence of hiv as a chronic condition me...,0,10
1,BACKGROUND,this paper describes the design and evaluation...,1,10
2,METHODS,this study is designed as a randomised control...,2,10
3,METHODS,the intervention group will participate in the...,3,10
4,METHODS,the program is based on self-efficacy theory a...,4,10
...,...,...,...,...
2211856,METHODS,"after crossover , the wait-list group received...",5,9
2211857,METHODS,performance was assessed by comparison to amer...,6,9
2211858,RESULTS,performance improved significantly after simul...,7,9
2211859,RESULTS,no improvement was detected as a function of c...,8,9
